# **Top2Vec 실습**

이번 노트북에서는 Top2Vec 모델을 한국어에 맞춰 사용할 수 있도록 Customizing한 후 인퍼런스 할 예정입니다. Top2Vec은 기본적으로 제공하는 시각화 툴이 없고 매우 불친절하므로, 해당 부분은 생략하도록 하겠습니다.

## **1 - 준비작업**

### **1.1 - 라이브러리 설치**
필요한 라이브러리를 설치합니다. 별도로 수정하지 말아주세요.

In [27]:
!pip install top2vec
!pip install konlpy

!pip install top2vec[sentence_encoders]
!pip install top2vec[sentence_transformers]
!pip install top2vec[indexing]

^C


### **1.2 - 라이브러리 불러오기**
필요한 라이브러리를 불러옵니다. 별도로 수정하지 말아주세요.

In [1]:
import os
import csv
import numpy as np
import random
import torch
import pandas as pd
from tqdm import tqdm

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from top2vec import Top2Vec
from sentence_transformers import SentenceTransformer

from konlpy.tag import Kkma, Komoran

from typing import List, Tuple, Dict, Literal, Optional, Callable

### **1.3 - Seed 고정**
결과를 일정하게 유지하기 위해 모든 SEED를 고정합니다. 별도로 수정하지 말아주세요.

In [2]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything()

### **1.4 - Colab**
코랩에 접근하기 위해 실행합니다. 그 전에, Colab 폴더에 코드와 데이터셋을 올바르게 넣었는지 확인해주세요.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

## **2 - 데이터셋**

### **2.1 - 데이터셋 로드**
데이터셋을 드라이브에서 불러옵니다.

In [4]:
PATH = './datasets/원자력+원전+탈원전_news_df.xlsx'
df1 = pd.read_excel(PATH)

PATH = './datasets/사설_반대.xlsx'
df2 = pd.read_excel(PATH)

PATH = './datasets/고리+신한울+새울+월성+신월성_news_df.xlsx'
df3 = pd.read_excel(PATH)

# df = pd.concat([df1, df2, df3], axis=0)

df = df2

미리보기

In [5]:
df.head()

,news_title,news_date,news_company,news_body
0,"[사설] `탈원전`, 방향은 맞지만 부작용 최소화해야",2017-05-29,디지털타임스,29일 국정기획자문위원회 이개호 경제2분과 위원장이 원자력안전위원회(원안위)에 원전...
1,[사설] 탈원전·석탄 조급증…안전하면서 값싼 전기는 없다,2017-05-30,한국경제,"문재인 정부의 에너지정책이 탈(脫)원전, 탈화력발전으로 급선회하고 있다. 그제 국정..."
2,[fn사설] 새 정부 탈원전 정책 우려 새겨 들어야,2017-06-01,파이낸셜뉴스,전문가 200명 비판 성명 방향 맞지만 너무 서둘러 에너지 분야 전문가들이 ...
3,[사설] 탈원전…정규직화…'밀어붙이기 정책'에서 놓치는 것들,2017-06-01,한국경제,"교수 230명 성명 ""에너지정책 전문가 논의 절실""일자리 해법, 누구나 비판하고 토..."
4,<사설>성급한 脫원전은 電力안보 해친다는 전문가들의 우려,2017-06-02,문화일보,문재인 정부의 탈(脫)원전 정책에 대해 에너지 전문가들이 정면으로 비판하고 나섰다....


### **2.2 - 텍스트 전처리**
해당 데이터는 크롤링된 데이터로, 올바른 결과를 위해서는 전처리가 필요합니다.
뉴스 기사를 위한 전처리 코드는 https://colab.research.google.com/drive/1O42v6YzHbTo_l36VG1xGp0aEuA0J8M3V?usp=sharing#scrollTo=P7JBkNna1jze 에서 가져왔습니다.

In [6]:
df['news_title'] = df['news_title'].map(lambda x: str(x))

In [7]:
import re, unicodedata
from string import whitespace
pattern_whitespace = re.compile(f'[{whitespace}]+')

df['news_title'] = df['news_title'].str.replace(pattern_whitespace, ' ').map(lambda x: unicodedata.normalize('NFC', x)).str.strip()


In [8]:
def clean_byline(text):
    # byline
    pattern_email = re.compile(r'[-_0-9a-z]+@[-_0-9a-z]+(?:\.[0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_url = re.compile(r'(?:https?:\/\/)?[-_0-9a-z]+(?:\.[-_0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_others = re.compile(r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$')
    result = pattern_email.sub('', text)
    result = pattern_url.sub('', result)
    result = pattern_others.sub('.', result)
    pattern_bracket = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\s)')
    result = pattern_bracket.sub('', result).strip()

    return result

df['news_title'] = df['news_title'].map(clean_byline)

In [9]:
from string import whitespace, punctuation

def text_filter(text):
    punct_except_percent = ''.join([chr for chr in punctuation if chr != '%'])
    whitespace_convert_pattern = re.compile(f'[{whitespace}{punct_except_percent}]+')
    exclude_pattern = re.compile(r'[^\% 0-9a-zA-Zㄱ-ㅣ가-힣]+')
    result = whitespace_convert_pattern.sub(' ', text)
    result = exclude_pattern.sub(' ', result).strip()
    result = whitespace_convert_pattern.sub(' ', result)
    return result

df['news_title'] = df['news_title'].map(text_filter)

전처리 후 결과 미리보기

In [10]:
df.head()

,news_title,news_date,news_company,news_body
0,탈원전 방향은 맞지만 부작용 최소화해야,2017-05-29,디지털타임스,29일 국정기획자문위원회 이개호 경제2분과 위원장이 원자력안전위원회(원안위)에 원전...
1,탈원전 석탄 조급증 안전하면서 값싼 전기는 없다,2017-05-30,한국경제,"문재인 정부의 에너지정책이 탈(脫)원전, 탈화력발전으로 급선회하고 있다. 그제 국정..."
2,새 정부 탈원전 정책 우려 새겨 들어야,2017-06-01,파이낸셜뉴스,전문가 200명 비판 성명 방향 맞지만 너무 서둘러 에너지 분야 전문가들이 ...
3,탈원전 정규직화 밀어붙이기 정책 에서 놓치는 것들,2017-06-01,한국경제,"교수 230명 성명 ""에너지정책 전문가 논의 절실""일자리 해법, 누구나 비판하고 토..."
4,성급한 원전은 안보 해친다는 전문가들의 우려,2017-06-02,문화일보,문재인 정부의 탈(脫)원전 정책에 대해 에너지 전문가들이 정면으로 비판하고 나섰다....


## **3 - 모델**

### **3.1 - 커스텀 토크나이저 설정**

Top2Vec에서 사용하는 Tokenizor는 Gensim의 simple prepreocessing을 사용하므로 띄어쓰기를 기준으로 토큰화를 진행합니다. 우리는 영어가 아닌 한국어를 모델링 할 것이므로, 띄어쓰기가 아닌 별도의 토크나이저를 사용해 토큰화하도록 하겠습니다.

토크나이저는 Kkma, Okt, Komoran 등 다양한 종류가 존재합니다. 가장 정확도가 높은 것은 Kkma이지만, 속도가 가장 빠른 komoran을 여기서는 사용하도록 하겠습니다.

불필요한 품사를 제거하고, 명사, 형용사, 동사, 관형사, 일반부사만 사용하도록 필터링하고, 불용어를 제거하는 함수를 제작합니다.

다른 모델과 달리 여기선 별도로 Vectorizer을 설정할 필요는 없습니다.

In [12]:
USER_DICT_PATH = './datasets/user_dic.txt'

custom_dic = [
    '탈원전\tNNG',
]

with open(USER_DICT_PATH, 'w', encoding='utf-8') as f:
    for line in custom_dic:
        f.write(line + '\n')

In [13]:
class CustomTokenizer:

    def __init__(
        self,
        tagger: Callable = Komoran(), # KoNLPy의 태거가 들어갑니다.
        stop_words: Optional[List[str]] = None, # 불용어 사전이 들어갑니다.
        tag_list: List[str] = ['NNG', 'NNP', 'NNB', 'VV', 'VA', 'MM', 'MAG']
        ):
      self.tagger = tagger
      self.stop_words = stop_words
      self.tag_list = tag_list

    def __call__(self, text):
      text = text[:10000]
      # 명사, 형용사, 동사, 관형사와 일반 부사만 취한다.
      word_tokens = list(map(lambda x: x[0], filter(lambda x: x[1] in self.tag_list, self.tagger.pos(text))))
      result = [word for word in word_tokens if word not in stop_words]
      return result

In [14]:
STOPWORDS_PATH = './datasets/user_stopwords.csv'

with open(STOPWORDS_PATH, 'r', encoding='cp949') as f:
    tmp = csv.reader(f)
    stop_words = set([l[0] for l in tmp])

tagger = Komoran(userdic='./datasets/user_dic.txt')
tokenizer = CustomTokenizer(tagger = tagger, stop_words = stop_words)

### **3.2 - 하이퍼파라미터 설정**

**설정해야하는 값**
- `min_count` :
    - 해당 값보다 적은 빈도를 보이는 토픽은 무시합니다.
    - 코퍼스가 작을 수록 이 값도 작아지는 것이 자연스럽습니다.
- `speed` : 
    - 모델이 얼마나 빠르게 Train할 것인지를 의미합니다.
    - 빠른 옵션은 낮은 퀄리티의 결과물을 생성합니다.
    - fast-learn, learn, deep-learn 중 하나를 고르면 됩니다.

In [15]:
min_count = 20
speed = 'learn'

## **4 - 모델 생성 & 토픽 추출하기**

모든 설정을 마쳤습니다. 이제 Top2Vec모델을 생성해보도록 하겠습니다. Top2Vec은 생성과 동시에 문서를 전달하여 토픽을 추출합니다.

이때, 문서는 string으로 이루어진 List형태로 전달해야합니다.

In [16]:
documents = df['news_title'].tolist()

model = Top2Vec(
    documents = documents, 
    embedding_model = 'universal-sentence-encoder-multilingual', 
    tokenizer = tokenizer,
    min_count = min_count,
    speed=speed
    )

2024-02-13 04:38:54,752 - top2vec - INFO - Pre-processing documents for training
c:\Users\owj04\anaconda3\envs\AIKU\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2024-02-13 04:38:56,203 - top2vec - INFO - Downloading universal-sentence-encoder-multilingual model
2024-02-13 04:39:27,577 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2024-02-13 04:39:31,117 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2024-02-13 04:39:43,748 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2024-02-13 04:39:43,807 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


`get_topic_sizes`으로 각 토픽에 해당하는 문장의 수, 즉 토픽의 크기를 알 수 있습니다.

`get_topics`으로 각 토픽에 해당하는 키워드를 알 수 있습니다.

두 정보를 하나의 데이터프레임에 합치도록하겠습니다.

In [48]:
topic_sizes, topic_nums = model.get_topic_sizes()
topic_words, word_scores, topic_nums = model.get_topics()


nr_topics = 4

result = pd.DataFrame.from_dict({
    'Topic': topic_nums,
    'Count': topic_sizes,
    'Representation': map(lambda x: x[:nr_topics],topic_words.tolist())
    })

In [51]:
result

,Topic,Count,Representation
0,0,317,"[탈원전, 한전, 중단, 의혹]"
1,1,152,"[정부, 공공, 민주당, 정권]"
2,2,117,"[전기, 전기료, 탈원전, 전력]"
3,3,104,"[경제, 위기, 민주당, 정부]"
4,4,103,"[한국, 탈원전, 세계, 나라]"
5,5,98,"[대통령, 민주당, 정치, 정권]"
6,6,94,"[정책, 정치, 탈원전, 정권]"
7,7,92,"[태양광, 원자력, 에너지, 풍력]"
8,8,91,"[에너지, 원자력, 전력, 풍력]"
9,9,83,"[감사원, 월성, 감사, 의혹]"


## **5 - 후처리**

## **5.1 -토픽 수 줄이기**
결과로 나온 토픽이 너무 많아 줄이고 싶을 수 있습니다. 아래와 같은 코드를 통해 자동으로 설정된 개수만큼 토픽 수를 줄일 수 있습니다.

In [74]:
model.hierarchical_topic_reduction(num_topics=10)

[[9, 21, 20, 0],
 [7, 11, 12, 22, 25, 8],
 [3, 6, 17, 26, 23],
 [29, 1],
 [15, 28, 4],
 [13, 2],
 [27, 10],
 [5],
 [16, 14],
 [19, 24, 18]]

In [76]:
topic_sizes, topic_nums = model.get_topic_sizes()
topic_words, word_scores, topic_nums = model.get_topics()

result = pd.DataFrame.from_dict({
    'Topic': topic_nums,
    'Count': topic_sizes,
    'Representation': map(lambda x: x[:nr_topics],topic_words.tolist()),
    'Scores': map(lambda x: x[:nr_topics],word_scores.tolist())
    })

In [77]:
result

,Topic,Count,Representation,Scores
0,0,317,"[탈원전, 한전, 중단, 의혹]","[0.5364928245544434, 0.21120133996009827, 0.20..."
1,1,152,"[정부, 공공, 민주당, 정권]","[0.5383365750312805, 0.323426753282547, 0.2435..."
2,2,117,"[전기, 전기료, 탈원전, 전력]","[0.47791701555252075, 0.44999611377716064, 0.3..."
3,3,104,"[경제, 위기, 민주당, 정부]","[0.5767709016799927, 0.19698600471019745, 0.19..."
4,4,103,"[한국, 탈원전, 세계, 나라]","[0.3720499575138092, 0.2620980143547058, 0.201..."
5,5,98,"[대통령, 민주당, 정치, 정권]","[0.5571733713150024, 0.3223887085914612, 0.308..."
6,6,94,"[정책, 정치, 탈원전, 정권]","[0.4897206425666809, 0.2886591851711273, 0.251..."
7,7,92,"[태양광, 원자력, 에너지, 풍력]","[0.5530035495758057, 0.2698877155780792, 0.266..."
8,8,91,"[에너지, 원자력, 전력, 풍력]","[0.5482548475265503, 0.37581774592399597, 0.34..."
9,9,83,"[감사원, 월성, 감사, 의혹]","[0.2750559449195862, 0.2658845782279968, 0.253..."


## **수고하셨습니다**

더 많은 사용법은 [공식 GitHub](https://top2vec.readthedocs.io/en/latest/Top2Vec.html#how-does-it-work)에서 찾아보세요!